In [ ]:
# FINAL SCRIPT 
import numpy as np, pandas as pd, time, tempfile, os
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR

#  1) composition-only dataset 
DATA_PATH = r"C:\Users\kagan\Downloads\COMPO_ONLY_model_ready_with_dSref_n0913.csv"  # <- kendi yoluna göre değiştir
df = pd.read_csv(DATA_PATH)
y = df["dS_ref"].astype(np.float32)
X = df.drop(columns=["dS_ref"]).astype(np.float32)

print("Dataset loaded:", X.shape, "target:", y.shape)
print("-"*60)

#  2) CV splitters: quantile-stratified (stabil R² için) 
# y'yi 5 dilime böl, aynı dağılımı her fold'a taşı
bins = pd.qcut(y, q=5, labels=False, duplicates='drop')
cv_for_grid = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_for_eval = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_list_grid = list(cv_for_grid.split(X, bins))
cv_list_eval = list(cv_for_eval.split(X, bins))

#  3) Pipelines (Scaler: sadece SVR). Cache açık.
cache_dir = tempfile.mkdtemp(prefix="sk_cache_")

pipe_rfr = Pipeline([('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))],
                    memory=cache_dir)

gbr_estimator = GradientBoostingRegressor(
    loss="huber", validation_fraction=0.1, n_iter_no_change=10, tol=1e-4, random_state=42
)
pipe_gbr = Pipeline([('regressor', gbr_estimator)], memory=cache_dir)

pipe_xgb = Pipeline([('regressor', xgb.XGBRegressor(objective='reg:squarederror',
                                                    random_state=42, n_jobs=-1))],
                    memory=cache_dir)

pipe_lgbm = Pipeline([('regressor', lgb.LGBMRegressor(random_state=42, n_jobs=-1))],
                     memory=cache_dir)

pipe_svr = Pipeline([('scaler', StandardScaler()),
                     ('regressor', SVR(kernel='rbf'))],
                    memory=cache_dir)

# 4) Hyperparameter grids  
param_grid_rfr = {
    'regressor__n_estimators': [100, 200, 400, 800],
    'regressor__max_depth': [10, 20, 30, None],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 5],
    'regressor__max_features': ['sqrt', 'log2']
}
param_grid_gbr = {
    'regressor__n_estimators': [100, 200, 400, 600, 800, 1000],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 5, 7, 9],
    'regressor__subsample': [0.6, 1.0],
    'regressor__max_features': ['sqrt', 'log2']
}
param_grid_xgb = {
    'regressor__n_estimators': [200, 400, 600, 800],
    'regressor__max_depth': [3, 5, 7, 9],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__subsample': [0.6, 1.0],
    'regressor__colsample_bytree': [0.6, 1.0]
}
param_grid_lgbm_extended = {
    'regressor__n_estimators': [200, 400, 600, 800],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__num_leaves': [20, 31, 40],
    'regressor__max_depth': [5, 10, -1],
    'regressor__subsample': [0.8, 1.0],
    'regressor__colsample_bytree': [0.8, 1.0]
}
param_grid_svr_extended = {
    'regressor__C': [1, 10, 100, 500],
    'regressor__gamma': [0.01, 0.1, 'scale']
}

models_to_run = {
    'Random Forest': (pipe_rfr, param_grid_rfr),
    'Gradient Boosting': (pipe_gbr, param_grid_gbr),
    'XGBoost': (pipe_xgb, param_grid_xgb),
    'LightGBM': (pipe_lgbm, param_grid_lgbm_extended),
    'SVR': (pipe_svr, param_grid_svr_extended)
}

# 5) Two-stage evaluation  
all_results = {}
scoring_metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

for model_name, (pipeline, params) in models_to_run.items():
    t0 = time.time()
    print(f"\n--- GridSearchCV for {model_name} ---")

    grid_search = GridSearchCV(
        estimator=pipeline, param_grid=params, scoring='r2',
        cv=cv_list_grid, n_jobs=-1, verbose=1
    )
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    print(f"Best params for {model_name}: {grid_search.best_params_}")

    print(f"--- cross_validate for {model_name} ---")
    cv_scores = cross_validate(best_model, X, y,
                               scoring=scoring_metrics,
                               cv=cv_list_eval, n_jobs=-1,
                               return_train_score=False)
    t1 = time.time()

    all_results[model_name] = {
        'mean_r2': cv_scores['test_r2'].mean(),
        'std_r2': cv_scores['test_r2'].std(),
        'mean_rmse': np.sqrt(-cv_scores['test_neg_mean_squared_error']).mean(),
        'std_rmse': np.sqrt(-cv_scores['test_neg_mean_squared_error']).std(),
        'mean_mae': -cv_scores['test_neg_mean_absolute_error'].mean(),
        'std_mae': cv_scores['test_neg_mean_absolute_error'].std(),
        'best_estimator': best_model,
        'time_taken': t1 - t0
    }
    print(f"Done {model_name} in {t1-t0:.1f}s")

#  6) Report 
final_df = pd.DataFrame(all_results).T.sort_values(by='mean_r2', ascending=False)
print("\n" + "="*60)
print("FINAL CONSOLIDATED MODEL COMPARISON (grids unchanged)")
print("="*60)
print(final_df[['mean_r2','std_r2','mean_rmse','std_rmse','mean_mae','std_mae']].round(3))


In [ ]:
# ANA KODUN EN SONUNA EKLENECEK KAYDETME BLOĞU
import pickle

print("\n" + "="*60)
print("Tüm model sonuçları 'final_model_results.pkl' dosyasına kaydediliyor...")

with open('final_model_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print("Kaydetme işlemi başarıyla tamamlandı.")
print("Artık bu sonuçları istediğiniz zaman yeniden yükleyebilirsiniz.")

In [ ]:
# DETAYLI ANALİZ BLOĞU
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.inspection import permutation_importance

# İSINSTANCE kontrolleri için:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb

# Gerekli değişkenlerin ortamda olduğunu kontrol et 
missing = []
for name in ["all_results", "X", "y"]:
    if name not in globals():
        missing.append(name)
if missing:
    raise RuntimeError(f"Gerekli değişken(ler) eksik: {missing}. Lütfen önce ana değerlendirme kodunu çalıştırın.")

print("Ana değerlendirme sonuçları ve veri bulundu.")

# Her model için analizleri ve grafikleri oluştur 
for model_name, result_data in all_results.items():

    print("\n" + "="*90)
    print(f"ANALYSIS FOR MODEL: {model_name}")
    print("="*90)

    # En iyi pipeline ve bileşenlerini al
    best_model_pipeline = result_data['best_estimator']

    # scaler opsiyonel olabilir (ağaçlarda yok)
    scaler = best_model_pipeline.named_steps.get('scaler', None)
    regressor = best_model_pipeline.named_steps.get('regressor', best_model_pipeline)

    # X'i SHAP/Permutation için hazırlama
    if scaler is not None:
        # SVR vb. için ölçekli X
        try:
            X_for_explain = pd.DataFrame(scaler.transform(X), columns=X.columns)
        except Exception as e:
            print(f"[Uyarı] Scaler.transform sırasında hata: {e}. Ham X kullanılacak.")
            X_for_explain = X.copy()
    else:
        # Ağaç modellerinde ham X
        X_for_explain = X.copy()

    # 1) Standard Feature Importance (ağaç modellerinde mevcut) 
    print(f"\n--- 1) Standard Feature Importance for {model_name} ---")
    if hasattr(regressor, "feature_importances_"):
        try:
            importances = regressor.feature_importances_
            fi_df = pd.DataFrame({"feature": X.columns, "importance": importances}) \
                        .sort_values("importance", ascending=False)

            plt.figure(figsize=(10, 8))
            sns.barplot(x='importance', y='feature', data=fi_df, palette='viridis')
            plt.title(f'Standard Feature Importance ({model_name})', fontsize=16)
            plt.xlabel('Importance Score', fontsize=12)
            plt.ylabel('Features', fontsize=12)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"[Atlandı] feature_importances_ çizimi sırasında hata: {e}")
    else:
        print(f"NOTE: '{model_name}' modelinde 'feature_importances_' özelliği yok (ör. SVR).")

    # 2) Permutation Feature Importance (pipeline üzerinde, R² düşüşü) 
    print(f"\n--- 2) Permutation Importance for {model_name} ---")
    try:
        perm = permutation_importance(
            best_model_pipeline, X, y,
            n_repeats=10, random_state=42, n_jobs=-1, scoring='r2'
        )
        sorted_idx = perm.importances_mean.argsort()

        fig, ax = plt.subplots(figsize=(12, 8))
        ax.boxplot(perm.importances[sorted_idx].T,
                   vert=False, labels=X.columns[sorted_idx])
        ax.set_title(f"Permutation Importance ({model_name})", fontsize=16)
        ax.set_xlabel("Performance Decrease (R²)", fontsize=12)
        ax.set_ylabel("Features", fontsize=12)
        ax.grid(axis='x', linestyle='--', alpha=0.4)
        fig.tight_layout()
        plt.show()
    except Exception as e:
        print(f"[Atlandı] Permutation importance sırasında hata: {e}")

    # 3) SHAP Analizi 
    print(f"\n--- 3) SHAP Analysis for {model_name} ---")
    try:
        # Ağaç modelleri: TreeExplainer
        if isinstance(regressor, (RandomForestRegressor,
                                  GradientBoostingRegressor,
                                  xgb.XGBRegressor,
                                  lgb.LGBMRegressor)):
            explainer = shap.TreeExplainer(regressor)
            shap_values = explainer.shap_values(X_for_explain)
            plt.figure()
            shap.summary_plot(shap_values, X_for_explain, show=False)
            plt.gcf().suptitle(f"SHAP Summary Plot ({model_name})", fontsize=16)
            plt.show()

        # SVR ve diğer modeller: KernelExplainer (pipeline.predict ile)
        elif isinstance(regressor, SVR):
            print("NOTE: Using KernelExplainer (daha yavaş). 100 örnek arka plan kullanılacak.")
            background = shap.sample(X_for_explain, 100, random_state=42)
            explainer = shap.KernelExplainer(best_model_pipeline.predict, background)
            shap_values = explainer.shap_values(X_for_explain, nsamples=100)  # hız için sınır
            plt.figure()
            shap.summary_plot(shap_values, X_for_explain, show=False)
            plt.gcf().suptitle(f"SHAP Summary Plot ({model_name})", fontsize=16)
            plt.show()
        else:
            # Diğer olası regressor tipleri için genel Explainer (yavaş olabilir)
            print("NOTE: Generic shap.Explainer kullanılıyor (yavaş olabilir).")
            explainer = shap.Explainer(best_model_pipeline.predict, X_for_explain)
            shap_values = explainer(X_for_explain)
            plt.figure()
            shap.summary_plot(shap_values, X_for_explain, show=False)
            plt.gcf().suptitle(f"SHAP Summary Plot ({model_name})", fontsize=16)
            plt.show()

    except Exception as e:
        print(f"[Atlandı] SHAP analizi sırasında hata: {e}")


In [ ]:
#  Final Comparison Plots (robust, annotated, matplotlib-only) 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) Model sırası 
order = ['Gradient Boosting', 'XGBoost', 'Random Forest', 'LightGBM', 'SVR']
df_plot = final_df.loc[order].copy()

# 2) Sayısal güvenlik
for c in ['mean_r2','std_r2','mean_rmse','std_rmse','mean_mae','std_mae']:
    df_plot[c] = pd.to_numeric(df_plot[c], errors='coerce')

# 3) Ortak renk paleti (viridis)
colors = plt.cm.viridis(np.linspace(0.12, 0.88, len(order)))

# 4) Figür
fig, axes = plt.subplots(1, 3, figsize=(22, 7))
fig.suptitle('Model Performance Comparison (10-fold Cross-Validation)', fontsize=22, y=1.03)

# Panel 1: R² 
axes[0].bar(order, df_plot['mean_r2'], yerr=df_plot['std_r2'],
            color=colors, edgecolor='black', capsize=8)
axes[0].set_title('Mean R² Score (Higher is better)', fontsize=16)
axes[0].set_ylabel('R² Score', fontsize=14)
axes[0].set_ylim(bottom=0)

#  Panel 2: RMSE 
axes[1].bar(order, df_plot['mean_rmse'], yerr=df_plot['std_rmse'],
            color=colors, edgecolor='black', capsize=8)
axes[1].set_title('Mean RMSE (Lower is better)', fontsize=16)
axes[1].set_ylabel('RMSE', fontsize=14)
axes[1].set_ylim(bottom=0)

# Panel 3: MAE 
axes[2].bar(order, df_plot['mean_mae'], yerr=df_plot['std_mae'],
            color=colors, edgecolor='black', capsize=8)
axes[2].set_title('Mean MAE (Lower is better)', fontsize=16)
axes[2].set_ylabel('MAE', fontsize=14)
axes[2].set_ylim(bottom=0)

# Ortak stil
for ax in axes:
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    ax.tick_params(axis='x', rotation=45, labelsize=12)
    ax.set_xlabel('')

plt.tight_layout()
plt.subplots_adjust(top=0.85)  # suptitle için boşluk
plt.savefig('model_performance_comparison_preferred.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# İSTATİSTİKSEL ANALİZ ve TASARIM KURALI DOĞRULAMA 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

#  Yardımcı fonksiyonlar 
def numeric_cols(df):
    return df.select_dtypes(include=[np.number]).columns.tolist()

def bootstrap_ci(stat_fn, y_true, y_pred, n_boot=5000, alpha=0.05, rng=42):
    """İkili sınıflandırma metrikleri için bootstrap GA (bias-corrected değil, hızlı)."""
    rs = np.random.RandomState(rng)
    stats = []
    n = len(y_true)
    idx = np.arange(n)
    for _ in range(n_boot):
        b = rs.choice(idx, size=n, replace=True)
        stats.append(stat_fn(y_true[b], y_pred[b]))
    lo = np.percentile(stats, 100*alpha/2)
    hi = np.percentile(stats, 100*(1-alpha/2))
    return float(np.mean(stats)), float(lo), float(hi)

def metrics_from_cm(cm):
    tn, fp, fn, tp = cm.ravel()
    acc = (tp+tn) / cm.sum()
    prec = tp / (tp+fp) if (tp+fp)>0 else 0.0
    rec  = tp / (tp+fn) if (tp+fn)>0 else 0.0  # TPR / sensitivity
    spec = tn / (tn+fp) if (tn+fp)>0 else 0.0 # TNR / specificity
    f1   = (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0
    # Matthews correlation
    denom = np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    mcc = ((tp*tn - fp*fn)/denom) if denom>0 else 0.0
    cov = (tp+fp)/cm.sum()  # kuralın "High" dediği pay (coverage)
    return dict(accuracy=acc, precision=prec, recall=rec, specificity=spec, f1=f1, mcc=mcc, coverage=cov)

def plot_confusion(cm, labels=('Actual Low','Actual High'), preds=('Predicted Low','Predicted High'), title='Confusion Matrix'):
    tn, fp, fn, tp = cm.ravel()
    totals = cm.sum()
    pct = cm / totals * 100.0
    annot = np.array([
        [f"TN\n{tn}\n{pct[0,0]:.1f}%", f"FP\n{fp}\n{pct[0,1]:.1f}%"],
        [f"FN\n{fn}\n{pct[1,0]:.1f}%", f"TP\n{tp}\n{pct[1,1]:.1f}%"]
    ])
    plt.figure(figsize=(7,6))
    ax = sns.heatmap(cm, annot=annot, fmt='', cmap='Blues', cbar=False,
                     xticklabels=preds, yticklabels=labels, linewidths=1, linecolor='black')
    ax.set_xlabel('Predicted by Rules', fontsize=12)
    ax.set_ylabel('Actual Performance', fontsize=12)
    ax.set_title(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# 1) Veriyi yükle 
print("Loading dataset for statistical analysis...")
DATA_PATH = r"C:\Users\kagan\Downloads\COMPO_ONLY_dataset.csv"
df = pd.read_csv(DATA_PATH)
print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns.")

#  2) Korelasyon ısı haritaları 
print("\nGenerating Correlation Heatmaps (numeric columns only)...")
num_cols = numeric_cols(df)
pearson_corr = df[num_cols].corr(method='pearson')
spearman_corr = df[num_cols].corr(method='spearman')

plt.figure(figsize=(12,10))
sns.heatmap(pearson_corr, annot=True, fmt=".2f", cmap='coolwarm', linewidths=.5)
plt.title('Pearson Correlation (numeric features)', fontsize=16)
plt.tight_layout(); plt.show()

plt.figure(figsize=(12,10))
sns.heatmap(spearman_corr, annot=True, fmt=".2f", cmap='viridis', linewidths=.5)
plt.title('Spearman Correlation (numeric features)', fontsize=16)
plt.tight_layout(); plt.show()

#  3) Gerçek performans etiketleri 
performance_threshold = 9.0
df['Actual Performance'] = np.where(df['dS'] >= performance_threshold, 1, 0)  # 1=High, 0=Low
print("\nCounts:", df['Actual Performance'].value_counts().to_dict())

#  4) Özellik dağılımları (kutu grafikleri) 
features_to_compare = [c for c in ['VEC','dHmix','dSmix'] if c in df.columns]
print("\nGenerating feature distribution plots...")
for feature in features_to_compare:
    plt.figure(figsize=(9,6))
    sns.boxplot(x='Actual Performance', y=feature, data=df, palette='viridis',
                order=[0,1])
    plt.title(f'Distribution of {feature} by Performance Group', fontsize=14)
    plt.xlabel('Actual Performance (0=Low, 1=High)', fontsize=12)
    plt.ylabel(feature, fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout(); plt.show()

#  5) Tasarım kuralı ve Confusion Matrix 
print("\nEvaluating fixed compositional design rules...")

#  kural sınırı yaz 
rule_VEC  = (df['VEC']  >= 2.40) & (df['VEC']  <= 7.30) if 'VEC'  in df.columns else True
rule_dSmix= (df['dSmix']>=11.50) & (df['dSmix']<=19.10) if 'dSmix'in df.columns else True
rule_dHmix= (df['dHmix']>=-46.5) & (df['dHmix']<=-27.6)  if 'dHmix' in df.columns else True

rule_mask = rule_VEC & rule_dSmix & rule_dHmix
df['Predicted Performance'] = np.where(rule_mask, 1, 0)

# Confusion matrix (label order: [Low, High] = [0,1])
cm = confusion_matrix(df['Actual Performance'], df['Predicted Performance'], labels=[0,1])
plot_confusion(cm, labels=('Actual Low','Actual High'), preds=('Predicted Low','Predicted High'),
               title='Confusion Matrix for Compositional Design Rules')

#  6) Metrikler + Bootstrap GA 
tn, fp, fn, tp = cm.ravel()
base_metrics = metrics_from_cm(cm)
print("\nPoint estimates:")
for k,v in base_metrics.items():
    print(f"  {k:>11s}: {v:.3f}")

# Bootstrap GA (accuracy, precision, recall, specificity, F1, MCC, coverage)
y_true = df['Actual Performance'].to_numpy()
y_pred = df['Predicted Performance'].to_numpy()

def stat_acc(y, yhat):  return ((y==yhat).mean())
def stat_prec(y, yhat): 
    tp = ((y==1)&(yhat==1)).sum(); fp = ((y==0)&(yhat==1)).sum()
    return tp/(tp+fp) if (tp+fp)>0 else 0.0
def stat_rec(y, yhat):
    tp = ((y==1)&(yhat==1)).sum(); fn = ((y==1)&(yhat==0)).sum()
    return tp/(tp+fn) if (tp+fn)>0 else 0.0
def stat_spec(y, yhat):
    tn = ((y==0)&(yhat==0)).sum(); fp = ((y==0)&(yhat==1)).sum()
    return tn/(tn+fp) if (tn+fp)>0 else 0.0
def stat_f1(y, yhat):
    p = stat_prec(y,yhat); r = stat_rec(y,yhat); 
    return (2*p*r)/(p+r) if (p+r)>0 else 0.0
def stat_mcc(y, yhat):
    tp = ((y==1)&(yhat==1)).sum(); tn = ((y==0)&(yhat==0)).sum()
    fp = ((y==0)&(yhat==1)).sum(); fn = ((y==1)&(yhat==0)).sum()
    denom = np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return ((tp*tn - fp*fn)/denom) if denom>0 else 0.0
def stat_cov(y, yhat): return (yhat==1).mean()

stats = {
    "accuracy":   stat_acc,
    "precision":  stat_prec,
    "recall_TPR": stat_rec,
    "specificity":stat_spec,
    "f1":         stat_f1,
    "mcc":        stat_mcc,
    "coverage":   stat_cov
}

print("\nBootstrap 95% CI (5000 resamples):")
for name, fn in stats.items():
    mean, lo, hi = bootstrap_ci(lambda yt, yp: fn(yt, yp), y_true, y_pred, n_boot=5000, alpha=0.05, rng=123)
    print(f"  {name:>11s}: {mean:.3f}  (95% CI: {lo:.3f}–{hi:.3f})")